# 要約 
このJupyter Notebookは、Kaggleのコンペティション「LLM 20 Questions」において、Gemmaモデルを使用するためのガイドラインを提供しています。初心者向けにステップバイステップでGemmaのセットアップと使用方法を説明しています。

### 問題の概要
本ノートブックは、Gemmaモデルを利用して「20の質問」ゲームをプレイするための準備作業に焦点を当てています。このゲームでは、プレイヤーがキーワードを思いつき、質問者が「はい」または「いいえ」で質問をし、キーワードを推測する形式です。Gemmaモデルを用いることで、より高度な推論や戦略的な質問が可能になります。

### アプローチと手法
ノートブックは以下の主要な手法とライブラリを使用しています：
1. **Gemmaモデルの選定と設定**: Gemmaモデルは、`GemmaForCausalLM`クラスを使用して設定され、特定のバリアント（例えば、2bや7b）に基づいています。ユーザーは使用するモデルのバリアントに応じて設定を変更できます。

2. **依存関係のインストール**: 必要なライブラリ（`immutabledict`や`sentencepiece`など）をインストールし、GemmaのPyTorch実装をダウンロードしてビルディングします。

3. **モデルのロードとトークナイザーの設定**: PyTorchとGemmaのコンフィギュレーションを活用してロードされたモデルにプロンプトを入力し、ユーザーが指定した設定（デバイスやモデルバリアントなど）に従ってモデルを初期化します。

4. **質問応答（QA）機能の実装**: `generate`メソッドを使って、モデルに対して質問を送信し、その回答を出力します。プロンプトエンジニアリング（指示チューニング）もサポートされており、特に「20の質問ゲーム」を意識したプロンプトの準備を行います。

### 使用されているライブラリとツール
- **PyTorch**: 深層学習モデルを扱うための主なライブラリ。
- **Gemma**: 質問応答や自然言語処理タスクに利用されるモデル。
- **Git**: Gemmaのコードをクローンするために使用される。

このノートブックは、Gemmaを「20の質問」ゲームで利用するための入門資料として非常に有用であり、読み手がGemmaの導入からモデルの活用までの過程を理解できるように構成されています。

---


# 用語概説 
以下は、提供されたJupyterノートブックに出てくる専門用語や特有のドメイン知識に関する説明です。機械学習・深層学習に関する基礎的な知識を持つ初心者を対象に、特に難しいと感じられる可能性のある用語に焦点を当てています。

1. **Gemmaモデル**: Googleが開発した言語モデルで、特定のタスク向けに設計されたさまざまなバリアントが存在します。このノートブックでは、特に「Causal LM」(因果言語モデル)としてのGemmaが紹介されています。

2. **Causal LM**: この用語は、モデルがこれまでの文脈を基に次の単語を生成することができるという意味です。つまり、前の単語が次の単語の生成に影響を与えるモデルのことを指します。

3. **トークナイザー**: テキストをコンピュータが理解できる形式（トークン）に変換するためのツールです。特に自然言語処理では、文章を単語やサブワード、文字などの単位に分割します。

4. **プロンプトエンジニアリング**: モデルに対して適切な入力（プロンプト）を作成する技術です。この技術により、モデルの応答の質を向上させることができます。

5. **指示チューニング**: モデルが特定の指示に従って予測や出力を生成する能力を高めるための手法です。これはプロンプトに特定の条件や文脈を与えることで行われます。

6. **温度 (temperature)**: モデルの生成する確率分布の「ランダム性」を制御するパラメータです。値が低いほど、モデルは高確率の単語を優先するようになり（より決定的な応答）、高いと多様な出力を生成します。

7. **top_p (核サンプリング)**: 出力の中で、累積確率がtop_pに達するまでの最も高い確率の単語群からサンプリングする手法です。これにより、候補の多様性を持ちながら適切な単語を選ぶことができます。

8. **top_k**: 予測する際に考慮する上位k個の単語を選択する手法です。これは、モデルの出力の中から最も確からしいk個を選択することで、生成されるテキストの質を保つ役割があります。

9. **重み (weights)**: ニューラルネットワーク内の各接続に付与されるパラメータであり、学習により最適化される数値です。重みは、モデルが入力に対して適切な出力を生成するための重要な要素です。

10. **評価モード (eval mode)**: モデルを評価する際の特定の状態で、通常はドロップアウトなどの特有の技術が無効になります。このモードは訓練済みモデルが正確に推論できるようにするために重要です。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

This notebook is for who is new to using Gemma in Kaggle.  
Let's go through how to use it step by step.

**Acknowledgment**  
This notebook based on [LLM 20 Questions Starter Notebook](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)

Before we start, check the GPU setup.

<img width="400px" src=attachment:9319be98-0139-41f9-a741-6b606b7cadc6.png>

---

## Add Gemma model

1. Click `+ Add Input` button.
<img src=attachment:6dabe5ed-9e7e-437b-be73-ee0c3b3a2afe.png width="400">

2. Search `Gemma` model.
<img src=attachment:68d9d1fa-84f4-4c37-acf9-f5cb2b825fcb.png width="400">

3. Set parameters. (I used *2b-it* because *7b* is too slow to test fast)
<img src=attachment:c7f1744e-08df-4ba7-ab61-4b0cd4e4e6fd.png width="400">

4. You can see the model has added.
<img src=attachment:056f294d-eee3-4bbb-b68b-6bc991a9c746.png width="400">

</div>
<div class="column-right">

# 日本語訳

このノートブックは、KaggleでGemmaを使用することに新しい方のためのものです。  
手順に従って、使い方を説明していきましょう。

**謝辞**  
このノートブックは、[LLM 20 Questions Starter Notebook](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)に基づいています。

始める前に、GPUのセットアップを確認してください。

<img width="400px" src=attachment:9319be98-0139-41f9-a741-6b606b7cadc6.png>

---

## Gemmaモデルを追加

1. `+ Add Input`ボタンをクリックします。
<img src=attachment:6dabe5ed-9e7e-437b-be73-ee0c3b3a2afe.png width="400">

2. `Gemma`モデルを検索します。
<img src=attachment:68d9d1fa-84f4-4c37-acf9-f5cb2b825fcb.png width="400">

3. パラメータを設定します。 (*2b-it*を使用しましたが、*7b*はテストを早く実行するには遅すぎるため)
<img src=attachment:c7f1744e-08df-4ba7-ab61-4b0cd4e4e6fd.png width="400">

4. モデルが追加されたのを確認できます。
<img src=attachment:056f294d-eee3-4bbb-b68b-6bc991a9c746.png width="400">


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%bash
# You can find input files in `/kaggle/input` directory.
tree /kaggle/input/gemma/pytorch
```

</div>
<div class="column-right">

# 日本語訳

```python
%%bash
# 入力ファイルは `/kaggle/input` ディレクトリにあります。
tree /kaggle/input/gemma/pytorch
```

</div>
</details>

In [ ]:
%%bash
# 入力ファイルは `/kaggle/input` ディレクトリにあります。
tree /kaggle/input/gemma/pytorch

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Setup Gemma to code

To use Gemma model, you have to setup few things.

## Install dependencies

</div>
<div class="column-right">

# 日本語訳

# Gemmaを使用するための設定

Gemmaモデルを使用するには、いくつかの設定が必要です。

## 依存関係のインストール


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%bash
# Install python packages in `/kaggle/working/submission/lib` directory.
mkdir -p /kaggle/working/submission/lib
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece

# Download `gemma_pytorch` and move it to `/kaggle/working/submission/lib` directory.
cd /kaggle/working
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/
```

</div>
<div class="column-right">

# 日本語訳

```python
%%bash
# `/kaggle/working/submission/lib` ディレクトリに Python パッケージをインストールします。
mkdir -p /kaggle/working/submission/lib
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece

# `gemma_pytorch`をダウンロードし、それを `/kaggle/working/submission/lib` ディレクトリに移動します。
cd /kaggle/working
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/
```

</div>
</details>

In [ ]:
%%bash
# `/kaggle/working/submission/lib` ディレクトリに Python パッケージをインストールします。
mkdir -p /kaggle/working/submission/lib
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece

# `gemma_pytorch`をダウンロードし、それを `/kaggle/working/submission/lib` ディレクトリに移動します。
cd /kaggle/working
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%bash
# You can see the result.
tree /kaggle/working/submission/lib
```

</div>
<div class="column-right">

# 日本語訳

```python
%%bash
# 結果を確認できます。
tree /kaggle/working/submission/lib
```

</div>
</details>

In [ ]:
%%bash
# 結果を確認できます。
tree /kaggle/working/submission/lib

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import sys

# Add module path to use libraries in `/kaggle/working/submission/lib` directory.
sys.path.insert(0, "/kaggle/working/submission/lib")
```

</div>
<div class="column-right">

# 日本語訳

```python
import sys

# ライブラリを使用するために、`/kaggle/working/submission/lib` ディレクトリにモジュールパスを追加します。
sys.path.insert(0, "/kaggle/working/submission/lib")
```

</div>
</details>

In [ ]:
import sys

# ライブラリを使用するために、`/kaggle/working/submission/lib` ディレクトリにモジュールパスを追加します。
sys.path.insert(0, "/kaggle/working/submission/lib")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Load Gemma model

</div>
<div class="column-right">

# 日本語訳

## Gemmaモデルのロード


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# Set global parameters
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
VARIANT = "2b-it"
# VARIANT = "7b-it-quant"
WEIGHTS_PATH = f"/kaggle/input/gemma/pytorch/{VARIANT}/2" # input model path

print("current device is", DEVICE)
print("current Gemma model variant is", VARIANT)
```

</div>
<div class="column-right">

# 日本語訳

```python
import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# グローバルパラメータを設定
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"  # 使用可能なGPUをチェック
VARIANT = "2b-it"
# VARIANT = "7b-it-quant"
WEIGHTS_PATH = f"/kaggle/input/gemma/pytorch/{VARIANT}/2" # 入力モデルパス

print("現在のデバイスは", DEVICE)
print("現在のGemmaモデルのバリアントは", VARIANT)
```

</div>
</details>

In [ ]:
import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

# グローバルパラメータを設定
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"  # 使用可能なGPUをチェック
VARIANT = "2b-it"
# VARIANT = "7b-it-quant"
WEIGHTS_PATH = f"/kaggle/input/gemma/pytorch/{VARIANT}/2" # 入力モデルパス

print("現在のデバイスは", DEVICE)
print("現在のGemmaモデルのバリアントは", VARIANT)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Set model_config
def get_model_config(variant):
    if "2b" in variant:
        return get_config_for_2b()
    if "7b" in variant:
        return get_config_for_7b()
    raise ValueError(f"Not supported variant - {variant}")
model_config = get_model_config(VARIANT)
model_config.tokenizer = f"{WEIGHTS_PATH}/tokenizer.model"
model_config.quant = "quant" in VARIANT
torch.set_default_dtype(model_config.get_dtype())

# Load Gemma model
gemma_model = GemmaForCausalLM(model_config)
gemma_model.load_weights(f"{WEIGHTS_PATH}/gemma-{VARIANT}.ckpt")
gemma_model = gemma_model.to(DEVICE).eval()
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデル設定を行います
def get_model_config(variant):
    if "2b" in variant:
        return get_config_for_2b()  # 2bモデルの設定を取得
    if "7b" in variant:
        return get_config_for_7b()  # 7bモデルの設定を取得
    raise ValueError(f"サポートされていないバリアント - {variant}")
model_config = get_model_config(VARIANT)  # 指定されたバリアントの設定を取得
model_config.tokenizer = f"{WEIGHTS_PATH}/tokenizer.model"  # トークナイザーのパスを設定
model_config.quant = "quant" in VARIANT  # バリアントに"quant"が含まれているか確認
torch.set_default_dtype(model_config.get_dtype())  # デフォルトのデータ型を設定

# Gemmaモデルを読み込む
gemma_model = GemmaForCausalLM(model_config)  # Causal LMモデルを初期化
gemma_model.load_weights(f"{WEIGHTS_PATH}/gemma-{VARIANT}.ckpt")  # 重みをロード
gemma_model = gemma_model.to(DEVICE).eval()  # モデルを指定デバイスに移動し評価モードに設定
```

</div>
</details>

In [ ]:
# モデル設定を行います
def get_model_config(variant):
    if "2b" in variant:
        return get_config_for_2b()  # 2bモデルの設定を取得
    if "7b" in variant:
        return get_config_for_7b()  # 7bモデルの設定を取得
    raise ValueError(f"サポートされていないバリアント - {variant}")
model_config = get_model_config(VARIANT)  # 指定されたバリアントの設定を取得
model_config.tokenizer = f"{WEIGHTS_PATH}/tokenizer.model"  # トークナイザーのパスを設定
model_config.quant = "quant" in VARIANT  # バリアントに"quant"が含まれているか確認
torch.set_default_dtype(model_config.get_dtype())  # デフォルトのデータ型を設定

# Gemmaモデルを読み込む
gemma_model = GemmaForCausalLM(model_config)  # Causal LMモデルを初期化
gemma_model.load_weights(f"{WEIGHTS_PATH}/gemma-{VARIANT}.ckpt")  # 重みをロード
gemma_model = gemma_model.to(DEVICE).eval()  # モデルを指定デバイスに移動し評価モードに設定

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Use Gemma

### Simple QA

You can use the model with `generate` method.

</div>
<div class="column-right">

# 日本語訳

## Gemmaの使用

### シンプルなQA

`generate`メソッドを使用してモデルを利用できます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
prompt = "Let me know how to use you."

model_answer = gemma_model.generate(
    prompts=prompt,
    device=torch.device(DEVICE),
    output_len=100, # max output tokens
    temperature=0.01,
    top_p=0.1,
    top_k=1,
)

print(model_answer)
```

</div>
<div class="column-right">

# 日本語訳

```python
prompt = "使い方を教えてください。"

model_answer = gemma_model.generate(
    prompts=prompt,
    device=torch.device(DEVICE),
    output_len=100, # 最大出力トークン数
    temperature=0.01,
    top_p=0.1,
    top_k=1,
)

print(model_answer)  # モデルの回答を出力
```

</div>
</details>

In [ ]:
prompt = "使い方を教えてください。"

model_answer = gemma_model.generate(
    prompts=prompt,
    device=torch.device(DEVICE),
    output_len=100, # 最大出力トークン数
    temperature=0.01,
    top_p=0.1,
    top_k=1,
)

print(model_answer)  # モデルの回答を出力

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### QA with instruction tuning

You can do the prompt engineering with instruction tuning.  
And you can see the code in detail in `GemmaFormatter` class of [Starter Notebook](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook).

Reference: https://ai.google.dev/gemma/docs/formatting

</div>
<div class="column-right">

# 日本語訳

### 指示チューニングによるQA

指示チューニングを使用してプロンプトエンジニアリングを行うことができます。  
詳細については、[Starter Notebook](https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook)の`GemmaFormatter`クラスを参照してください。

参考: https://ai.google.dev/gemma/docs/formatting


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
system_prompt = "You are an AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific person, place, or thing."

prompt = "<start_of_turn>user\n" # GemmaFormatter.user
prompt += f"{system_prompt}<end_of_turn>\n"
prompt += "<start_of_turn>model\n" # GemmaFormatter.start_model_turn

print(prompt)
```

</div>
<div class="column-right">

# 日本語訳

```python
system_prompt = "あなたは20質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを思いつき、質問者がはいまたはいいえの質問に答えます。キーワードは特定の人物、場所、または物です。"

prompt = "<start_of_turn>user\n" # GemmaFormatter.user
prompt += f"{system_prompt}<end_of_turn>\n"  # システムプロンプトを追加
prompt += "<start_of_turn>model\n" # GemmaFormatter.start_model_turn

print(prompt)  # プロンプトを出力
```

</div>
</details>

In [ ]:
system_prompt = "あなたは20質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを思いつき、質問者がはいまたはいいえの質問に答えます。キーワードは特定の人物、場所、または物です。"

prompt = "<start_of_turn>user\n" # GemmaFormatter.user
prompt += f"{system_prompt}<end_of_turn>\n"  # システムプロンプトを追加
prompt += "<start_of_turn>model\n" # GemmaFormatter.start_model_turn

print(prompt)  # プロンプトを出力

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model_answer = gemma_model.generate(
    prompts=prompt,
    device=torch.device(DEVICE),
    output_len=100, # max output tokens
    temperature=0.01,
    top_p=0.1,
    top_k=1,
)

print(model_answer)
```

</div>
<div class="column-right">

# 日本語訳

```python
model_answer = gemma_model.generate(
    prompts=prompt,
    device=torch.device(DEVICE),
    output_len=100, # 最大出力トークン数
    temperature=0.01,
    top_p=0.1,
    top_k=1,
)

print(model_answer)  # モデルの回答を出力
```

</div>
</details>

In [ ]:
model_answer = gemma_model.generate(
    prompts=prompt,
    device=torch.device(DEVICE),
    output_len=100, # 最大出力トークン数
    temperature=0.01,
    top_p=0.1,
    top_k=1,
)

print(model_answer)  # モデルの回答を出力

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

---

I hope now you can use how to use the Gemma model for the competition  
and this notebook helps understand the starter notebook.

</div>
<div class="column-right">

# 日本語訳

---

これで、Gemmaモデルをコンペティションで利用する方法が理解できたことを願っています。  
このノートブックがスターターノートブックを理解するのに役立ちますように。

</div>

---

これで、Gemmaモデルをコンペティションで利用する方法が理解できたことを願っています。  
このノートブックがスターターノートブックを理解するのに役立ちますように。